In [ ]:
# DF model used for non-defended dataset
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers import ELU
from keras.initializers import glorot_uniform

In [ ]:
class DFNet: # 신경망 모델을 정의하는 클래스
  @staticmethod # 이 메서드가 인스턴스가 아닌 클래스에서 직접 호출될 수 있음을 의미
  def build(input_shape, classes):
    model = Sequential()

    #Block1
    #Block1에서는 Conv1D 계층을 두 번 쌓고, 풀링 계층과 드롭아웃을 적용
    filter_num=['None', 32, 64, 128, 256] # filter_num개의 필터(특성 맵)를 사용하여 데이터를 학습
    kernel_size=['None',8,8,8,8] # 1차원 필터의 크기
    conv_stride_size=['None',1,1,1,1]
    pool_stride_size=['None',4,4,4,4]
    pool_size=['None',8,8,8,8]

    model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1], input_shape=input_shape,
                     strides=conv_stride_size[1], padding='same',
                     name='block1_conv1'))
    model.add(BatchNormalization(axis=-1)) # 각 계층의 출력값을 정규화하여 학습 속도를 높이고 과적합 방지.
    model.add(ELU(alpha=1.0, name='block1_adv_act1')) # 활성화 함수
    model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1],
                     strides=conv_stride_size[1], padding='same',
                     name='block1_conv2'))
    model.add(BatchNormalization(axis=-1))
    model.add(ELU(alpha=1.0, name='block_adv_act2'))

    # MaxPooling1D: 최대 풀링을 사용해 특성 맵의 크기를 줄임.
    # 최대 풀링: CNN에서 자주 사용되는 특정 영역에서 가장 큰 값만을 선택하는 방식의 다운샘플링 방법.
    model.add(MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1], padding='same', name='block1_pool'))
    # Dropout(0,1): 과적합을 방지하기 위해 일부 뉴런을 무작위로 비활성화.
    model.add(Dropout(0.1, name='block1_dropout'))


    #Block2,3,4
    #각 블록은 2번의 Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout 계층을 포함.
    #활성화 함수로는 ELU 혹은 ReLU가 사용되며, 점진적으로 더 깊은 특성 맵을 학습.

    #Block2
    model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                     strides=conv_stride_size[2], padding='same',
                     name='block2_conv1'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='block2_act1'))

    model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                     strides=conv_stride_size[2], padding='same',
                     name='block_conv2'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='block2_act2'))

    model.add(MaxPooling1D(pool_size=pool_size[2], strides=pool_stride_size[2],
                           padding='same', name='block2_pool'))
    model.add(Dropout(0.1, name='block2_dropout'))

    #Block3
    model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                     strides=conv_stride_size[3], padding='same',
                     name='block3_conv1'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='block3_act1'))

    model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                     strides=conv_stride_size[3], padding='same',
                     name='block3_conv2'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='block3_act2'))

    model.add(MaxPooling1D(pool_size=pool_size[3], strides=pool_stride_size[3],
                           padding='same', name='block3_pool'))
    model.add(Dropout(0.1, name='block3_dropout'))

    #Block4
    model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                     strides=conv_stride_size[4], padding='same',
                     name='block4_conv1'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='block4_act1'))

    model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                     strides=conv_stride_size[4], padding='same',
                     name='block4_conv2'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='block4_act2'))

    model.add(MaxPooling1D(pool_size=pool_size[4], strides=pool_stride_size[4],
                           padding='same', name='block4_pool'))
    model.add(Dropout(0.1, name='block4_dropout'))



    #Fully Connected Layers
    model.add(Flatten(name='flatten')) # Flatten 계층은 3D 출력 데이터를 1D 벡터로 변환하여 밀집 계층에 입력할 수 있도록 준비.
    model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='fc1')) # 512개의 뉴런을 가진 전결합 층을 정의 및 Glorot uniform으로 가중치 초기화
    model.add(BatchNormalization())
    model.add(Activation('relu', name='fc1_act'))
    model.add(Dropout(0.7, name='fc1_dropout'))

    model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='fc2'))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='fc2_act'))
    model.add(Dropout(0.5, name='fc2_dropout'))

    model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc3')) # Dense(classes)는 출력 계층으로, classes 만큼의 출력을 생성
    model.add(Activation('softmax', name='softmax'))
    return model